In [171]:
from lxml import html
from bs4 import BeautifulSoup
import requests
import re

In [172]:
main_url = 'https://ro.wikipedia.org'
url = main_url + '/wiki/Categorie:Liste_de_localități_din_România'
page = requests.get(url, timeout=1)

In [173]:
# print(page.content)

In [174]:
soup = BeautifulSoup(page.text, "html.parser")
# print(soup.prettify)

In [175]:
all_links = soup.find_all("div", class_="mw-category")[0].find_all("a")
pages_links = []
for l in all_links:
    if "județul" in l.text:
        pages_links.append(l.get("href"))
        
print(len(pages_links))

41


In [176]:
def parse_jud(soup_jud):
    title = soup_jud.find("h1", id="firstHeading").text
#     print(title)
    jud_name = title.split("județul ", 1)[1]
    return jud_name

In [177]:
# jud_url = "https://ro.wikipedia.org/wiki/List%C4%83_de_localit%C4%83%C8%9Bi_din_jude%C8%9Bul_Satu_Mare"
# page_jud = requests.get(jud_url, timeout=1)
# soup_jud = BeautifulSoup(page_jud.text, "html.parser")

In [178]:
# title = soup_jud.find("h1", id="firstHeading").text
# print(title)

In [ ]:
# jud = re.match(r'județul (\w+)', title, re.I|re.M)

In [ ]:
rows_list = []

for jud_page in pages_links:
    jud_url = main_url+jud_page
    page_jud = requests.get(jud_url, timeout=1)
    soup_jud = BeautifulSoup(page_jud.text, "html.parser")
    jud_name = parse_jud(soup_jud)
    tabl = soup_jud.find("table", class_="wikitable sortable")
    for row in tabl.findAll("tr"):
        cell = row.find("td")
        if cell:
            cell_link = cell.find("a")
            if cell_link:
                nume_localitate = cell_link.text
                rows_list.append({'Judet':jud_name, 'Localitate':nume_localitate})

import pandas as pd
x = pd.DataFrame(rows_list)
print(x.size)


In [ ]:
print(x.loc[13000:20000])

In [ ]:
x.to_csv("localitati_ro_big.csv")